In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory
from keras.utils.np_utils import to_categorical
import os
print(os.listdir("../input"))
from keras.models import Model
from keras.layers import LSTM,Embedding,Input,Flatten,Dot,Concatenate,RepeatVector,Activation,Dense
# Any results you write to the current directory are saved as output.

Using TensorFlow backend.


['repository']


In [2]:
f=open("../input/repository/udacity-deep-learning-c6b46a0/seq2seq/data/letters_source.txt")
data=f.read()

In [3]:
data=data.split("\n")

In [4]:
sourcedata=open("../input/repository/udacity-deep-learning-c6b46a0/seq2seq/data/letters_target.txt")

In [5]:
source=sourcedata.read()
source=source.split("\n")

In [6]:
source[:5]

['abqqs', 'npy', 'bjluw', 'bqv', 'aikl']

In [7]:
data[:5]

['bsaqq', 'npy', 'lbwuj', 'bqv', 'kial']

In [8]:
data1=[list(i) for i in data]
source1=[list(i) for i in source]

In [9]:
data1[0]

['b', 's', 'a', 'q', 'q']

In [10]:
source1[0]

['a', 'b', 'q', 'q', 's']

In [11]:
uniquewords=set()
for i in data1:
    uniquewords.update(set(i))

In [12]:
uniquewords

{'a',
 'b',
 'c',
 'd',
 'e',
 'f',
 'g',
 'h',
 'i',
 'j',
 'k',
 'l',
 'm',
 'n',
 'o',
 'p',
 'q',
 'r',
 's',
 't',
 'u',
 'v',
 'w',
 'x',
 'y',
 'z'}

In [13]:
wordindex=dict([(v,k+1) for k,v in enumerate(uniquewords)])

In [14]:
wordindex

{'p': 1,
 'q': 2,
 'v': 3,
 'm': 4,
 's': 5,
 'h': 6,
 'd': 7,
 'a': 8,
 'r': 9,
 'f': 10,
 'j': 11,
 'b': 12,
 'y': 13,
 'e': 14,
 'u': 15,
 'g': 16,
 'x': 17,
 'z': 18,
 'i': 19,
 't': 20,
 'l': 21,
 'n': 22,
 'o': 23,
 'c': 24,
 'w': 25,
 'k': 26}

In [15]:
wordindex['<PAD>']=0

In [16]:
wordindex['<go>']=len(wordindex)


In [17]:
indexword=dict([(k,v) for v,k in wordindex.items()])

In [18]:
indexword

{1: 'p',
 2: 'q',
 3: 'v',
 4: 'm',
 5: 's',
 6: 'h',
 7: 'd',
 8: 'a',
 9: 'r',
 10: 'f',
 11: 'j',
 12: 'b',
 13: 'y',
 14: 'e',
 15: 'u',
 16: 'g',
 17: 'x',
 18: 'z',
 19: 'i',
 20: 't',
 21: 'l',
 22: 'n',
 23: 'o',
 24: 'c',
 25: 'w',
 26: 'k',
 0: '<PAD>',
 27: '<go>'}

In [19]:
len1=len(wordindex)

In [20]:
#preprocessing ,padding 
maxdatalen=max(map(len,data1))

In [21]:
maxsourcelen=max(map(len,source1))

In [22]:
maxsourcelen

7

In [23]:
wordindex['<go>']

27

In [24]:
xtrain=np.zeros((len(data1),maxdatalen))
y=np.zeros((len(source1),maxsourcelen+1))

In [25]:
for i in range(len(data1)):
    m=len(data1[i])
    xtrain[i,:m]=[wordindex[j] for j in data[i]]
    y[i,:m+1]=[wordindex['<go>']]+[wordindex[j] for j in source[i]]

In [26]:
y[:5]

array([[27.,  8., 12.,  2.,  2.,  5.,  0.,  0.],
       [27., 22.,  1., 13.,  0.,  0.,  0.,  0.],
       [27., 12., 11., 21., 15., 25.,  0.,  0.],
       [27., 12.,  2.,  3.,  0.,  0.,  0.,  0.],
       [27.,  8., 19., 26., 21.,  0.,  0.,  0.]])

In [27]:
xtrain[:5]

array([[12.,  5.,  8.,  2.,  2.,  0.,  0.],
       [22.,  1., 13.,  0.,  0.,  0.,  0.],
       [21., 12., 25., 15., 11.,  0.,  0.],
       [12.,  2.,  3.,  0.,  0.,  0.,  0.],
       [26., 19.,  8., 21.,  0.,  0.,  0.]])

In [28]:
Tx=7
Ty=8

In [29]:
y[:5]

array([[27.,  8., 12.,  2.,  2.,  5.,  0.,  0.],
       [27., 22.,  1., 13.,  0.,  0.,  0.,  0.],
       [27., 12., 11., 21., 15., 25.,  0.,  0.],
       [27., 12.,  2.,  3.,  0.,  0.,  0.,  0.],
       [27.,  8., 19., 26., 21.,  0.,  0.,  0.]])

In [30]:
y_hot=to_categorical(y,len(wordindex))

In [31]:
y_hot.shape

(10000, 8, 28)

In [32]:
densor=Dense(16,activation='relu')
densor1=Dense(1,activation='relu')
repeat=RepeatVector(Tx)
activator=Activation('softmax')
dotor=Dot(axes=1)

In [33]:
def one_step_attention(a,st_prev):
    st_prev1=repeat(st_prev)  # m X Tx X n_s
    concat=Concatenate(axis=-1)([a,st_prev1])
    dens=densor(concat)
    dens1=densor1(dens)  #weights for attention  m X Tx
    context=dotor([dens1,a])
    context=activator(context)
    return context

In [34]:
n_a=64
n_s=64
post_activation_lstm=LSTM(n_s,return_state=True)
outputlayer=Dense(len(wordindex),activation='softmax')

In [35]:
Tx

7

In [36]:
x=Input(shape=(Tx,))
embed=Embedding(len1,24,input_length=Tx)(x)

In [37]:
s0=Input(shape=(n_s,))
c0=Input(shape=(n_s,))
s=s0
c=c0

In [38]:
outputs=[]
a=LSTM(n_a,return_sequences=True)(embed)

In [39]:
for i in range(Ty):
    context=one_step_attention(a,s)
    s,_,c=post_activation_lstm(context,initial_state=[s,c])
    
    out=outputlayer(s)
    outputs.append(out)
    
model=Model(inputs=[x,s0,c0],outputs=outputs)

In [40]:
outputs[0]

<tf.Tensor 'dense_3/Softmax:0' shape=(?, 28) dtype=float32>

In [41]:
len(outputs)

8

In [42]:
y_hot[0].shape

(8, 28)

In [43]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 7)            0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 7, 24)        672         input_1[0][0]                    
__________________________________________________________________________________________________
input_2 (InputLayer)            (None, 64)           0                                            
__________________________________________________________________________________________________
lstm_2 (LSTM)                   (None, 7, 64)        22784       embedding_1[0][0]                
__________________________________________________________________________________________________
repeat_vec

In [44]:
model.compile(optimizer='adam',metrics=['accuracy'],loss='categorical_crossentropy')

In [45]:
m=len(data1)
epochs=100


In [46]:
s0=np.zeros((m,n_s))
c0=np.zeros((m,n_s))

In [47]:
xtrain.shape

(10000, 7)

In [48]:
len(list(y_hot.swapaxes(0,1)))

8

In [49]:
model.fit([xtrain,s0,c0],list(y_hot.swapaxes(0,1)),epochs=epochs,batch_size=64)

Epoch 1/100
10000/10000 [==============================] - 18s 2ms/step - loss: 19.7215 - dense_3_loss: 1.1064 - dense_3_acc: 0.0909 - dense_3_acc_1: 0.0000e+00 - dense_3_acc_2: 0.1365 - dense_3_acc_3: 0.2833 - dense_3_acc_4: 0.4150 - dense_3_acc_5: 0.5598 - dense_3_acc_6: 0.7026 - dense_3_acc_7: 0.8436
Epoch 2/100
10000/10000 [==============================] - 9s 911us/step - loss: 16.2069 - dense_3_loss: 0.4561 - dense_3_acc: 0.8930 - dense_3_acc_1: 0.0000e+00 - dense_3_acc_2: 0.1383 - dense_3_acc_3: 0.2868 - dense_3_acc_4: 0.4226 - dense_3_acc_5: 0.5692 - dense_3_acc_6: 0.7130 - dense_3_acc_7: 0.8550
Epoch 3/100
10000/10000 [==============================] - 9s 912us/step - loss: 13.3604 - dense_3_loss: 0.3300 - dense_3_acc: 1.0000 - dense_3_acc_1: 0.0000e+00 - dense_3_acc_2: 0.1646 - dense_3_acc_3: 0.3495 - dense_3_acc_4: 0.4907 - dense_3_acc_5: 0.6269 - dense_3_acc_6: 0.7562 - dense_3_acc_7: 0.8798
Epoch 4/100
10000/10000 [==============================] - 9s 924us/step - loss: 10

In [50]:
s1=np.zeros((10,n_s))
c1=np.zeros((10,n_s))

In [51]:
# During training i realised  accuracy of first time step of decoder changes less as compared to other timesteps
# so its better if we add <go> token at start of decoder train output and then train model

# After watching training process :
# Initially last timesteps of decoder improves a lot faster then first time steps of decoder 
# later on first few times of decoder starts to improve

In [52]:
out1=model.predict([xtrain[:10],s1,c1])

In [53]:
out2=np.stack(out1,axis=1)

In [54]:
out2.shape

(10, 8, 28)

In [55]:
outpred=np.argmax(out2,axis=-1)

In [56]:
outpred

array([[27,  8, 12,  2,  2,  5,  0,  0],
       [27, 22,  1, 13,  0,  0,  0,  0],
       [27, 12, 11, 21, 15, 25,  0,  0],
       [27, 12,  2,  3,  0,  0,  0,  0],
       [27,  8, 19, 26, 21,  0,  0,  0],
       [27, 12, 24, 14,  4, 20,  0,  0],
       [27,  7, 14, 16, 11,  1,  2, 17],
       [27, 22,  1,  5,  3,  0,  0,  0],
       [27,  6, 26, 23, 15, 18,  0,  0],
       [27, 24, 26, 21,  4,  2,  0,  0]])

In [57]:
y[:10]

array([[27.,  8., 12.,  2.,  2.,  5.,  0.,  0.],
       [27., 22.,  1., 13.,  0.,  0.,  0.,  0.],
       [27., 12., 11., 21., 15., 25.,  0.,  0.],
       [27., 12.,  2.,  3.,  0.,  0.,  0.,  0.],
       [27.,  8., 19., 26., 21.,  0.,  0.,  0.],
       [27.,  8.,  7.,  7.,  4., 20.,  0.,  0.],
       [27.,  7., 14., 16., 11.,  1.,  1., 17.],
       [27., 22.,  1.,  5.,  3.,  0.,  0.,  0.],
       [27.,  6., 21., 23., 15., 18.,  0.,  0.],
       [27., 24., 26., 21.,  4.,  2.,  0.,  0.]])

In [58]:
#Done 